# CHALLENGE TELECOM X - PARTE 1

**Objetivo: Investigar o CHURN DE CLIENTES**

A Telecom X é uma empresa de telecomunicações e a equipe de TI precisa analisar o Churn de Clientes, que está elevado.

## 1 - Buscar, tratar e realizar uma análise exploratoria dos dados de cliente.

A) Buscar a base de dados com as informações originais, diretamente da API.

B) Fazer uma análise exploratória inicial.

C) Identificar dados nulos e fazer o tratamento.


### 1.1 - Carregar Base de Dados.

Arquivo está no formato JSON.

Primeiro, carregar as bibliotecas que serão utilizadas no projeto.

In [ ]:
# Importar bibliotecas a serem utilizadas no projeto.
import pandas as pd
import json
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import pandas as pd
import os

Carregar a base de dados, através de referenciamento ao endereço no GitHub.

Tratar o aninhamento de colunas.

In [ ]:
# Fazer a primeira carga dos dados.
caminho_base = Path.cwd().parent
origem = caminho_base / "data" / "raw" / "TelecomX_Data.JSON"
dados = pd.read_json(origem)
dados.head()

In [ ]:
# Normalização (desaninhar) mantendo "index", "customerID", "Churn" como metadados
dados_normal = pd.json_normalize(
    dados.to_dict(orient='records'),
    meta = ["index", "customerID", "Churn"],
    sep="_"
)

dados_normal.head()

### 1.2 - DataFrame normalizado, fazer a exploração inicial dos dados.

Verificar:

* valores nulos;

* valores em branco (vazios);

* outros possíveis erros (duplicados, inconsistências)

#### 1.2.1 - **Dicionário de dados original.**
*   customerID: número de identificação único de cada cliente
*   Churn: se o cliente deixou ou não a empresa
*   gender: gênero (masculino e feminino)
*   SeniorCitizen: informação sobre um cliente ter ou não idade igual ou maior que 65 anos
*   Partner: se o cliente possui ou não um parceiro ou parceira
*   Dependents: se o cliente possui ou não dependentes
*   tenure: meses de contrato do cliente
*   PhoneService: assinatura de serviço telefônico
*   MultipleLines: assinatura de mais de uma linha de telefone
*   InternetService: assinatura de um provedor internet
*   OnlineSecurity: assinatura adicional de segurança online
*   OnlineBackup: assinatura adicional de backup online
*   DeviceProtection: assinatura adicional de proteção no dispositivo
*   TechSupport: assinatura adicional de suporte técnico, menos tempo de espera
*   StreamingTV: assinatura de TV a cabo
*   StreamingMovies: assinatura de streaming de filmes
*   Contract: tipo de contrato
*   PaperlessBilling: se o cliente prefere receber online a fatura
*   PaymentMethod: forma de pagamento
*   Charges.Monthly: total de todos os serviços do cliente por mês
*   Charges.Total: total gasto pelo cliente

#### 1.2.2 - Verificar valores únicos por coluna.


In [ ]:
# Conferência de valores únicos por coluna.
print("\n--- Valores únicos por coluna ---")

for coluna in dados_normal.columns:
    valores_unicos = dados_normal[coluna].nunique()
    print(f'\n* Coluna "{coluna}": {valores_unicos}')
    if valores_unicos < 10:
      print(dados_normal[coluna].unique())

    print("-" * 60)

#### 1.2.3 - Verificar Linhas Duplicadas.

In [ ]:
# Verificar se existem linhas duplicadas.
duplicados = dados_normal.duplicated().sum()
print(f"--> Total de linhas COMPLETAMENTE duplicadas: {duplicados}\n")


#### 1.2.4 - Checar Valores Nulos.

In [ ]:
# Checar valores nulos por coluna.
valores_nulos = dados_normal.isnull().values.sum()
print(f"--> Total valores nulos por coluna: {valores_nulos}\n")

#### 1.2.5 - Verificar Valores em Branco, Vazios ou Espaços.

In [ ]:
print("Checar Valores em Branco, Vazios ou Espaço por Coluna:\n")

for coluna in dados_normal.columns:
    # Verifica apenas colunas do tipo 'object' (texto)
    if dados_normal[coluna].dtype == "object":
        vazios = (dados_normal[coluna].str.strip() == "").sum()

        # Mostra apenas se houver strings vazias
        if vazios > 0:
            total = len(dados_normal[coluna])
            percentual_vazios = (vazios / total) * 100
            print(f"Coluna '{coluna}':")
            print(f"- Strings vazias (''): {vazios} ({percentual_vazios:.2f}%)\n")

### 1.3 - **Dar tratamento às inconsistências identificadas.**

In [ ]:
# Transformar a coluna "account_Charges_Total" de string para float.

dados_normal["account_Charges_Total"] = pd.to_numeric(
    dados_normal["account_Charges_Total"],
    errors = "coerce"
)


print(f'Coluna "account_Charges_Total": {dados_normal["account_Charges_Total"].dtype}.')

In [ ]:
# Conferir como ficaram os espaços em branco, depois da conversão.
ex_espacos = dados_normal[dados_normal["account_Charges_Total"].astype(str).str.strip() == ""]
print(f"Ex-espaços em branco:\n{ex_espacos[['account_Charges_Total']]}")
print(f"\nTotal: {len(ex_espacos)} → Convertidos para NaN")

In [ ]:
# Excluir as linhas em branco, na coluna "Churn"
print(f"Linhas antes: {len(dados_normal)}")

dados_normal = dados_normal[~dados_normal["Churn"].str.strip().isin(["", " "])]

print(f"Linhas depois: {len(dados_normal)}")

In [ ]:
# Conferir se há algum campo em branco ou com espaço.
campos_vazios = dados_normal.apply(
    lambda col: col.astype(str).str.strip().isin(["", " "]).sum()
)

print(campos_vazios)

### 1.4 - Incluir coluna Charges Daily.

*Critério:* (Charges_Monthly / 30)

Usar a coluna de Charges_Total pode trazer distorções para os contratos Long Term, já que carregam a somatória de todo o período do contrato, com possíveis situações diferenciadas do período atual em análise.

In [ ]:
# Incluir coluna de contas diárias (Charges Daily)
dados_normal.insert(loc=19,
                    column="account_Charges_Daily",
                    value=round(dados_normal["account_Charges_Monthly"] / 30, 2))

print("\nColunas após inserção:")
print(dados_normal.columns.tolist())
print("\nPrimeiros valores da nova coluna:")
print(dados_normal["account_Charges_Daily"].head())

In [ ]:
# Conferir como ficou o dataframe nesta parte.
print(dados_normal.info())


### 1.5 - Corrigir coluna "account_Charges_Total.

Coluna "account_Charges_Total" está com 7.032 registros, 11 a menos que as outras colunas do dataframe.

Verificar o que ocorreu, e consertar.

*Hipótese*: são clientes novos, que entraram na Telecom X no período em análise, já que o tempo de permanência desses cliente ainda é 0 meses. Se o tempo de permanência fosse em dias, a dedução seria mais fácil.
Poderíamos replicar o valor mensal para o valor total que estaria correto, mas a base de permanência seria diferente e teríamos uma variável com 0, e em algum momento pode gerar um erro.

**Ação**: remover as linhas com Charges_Total igual a 0, já que não terão impacto significativo nas previsões a serem realizadas.

In [ ]:
# Filtrar linhas onde 'account_Charges_Total' é nulo
linhas_faltantes = dados_normal[dados_normal["account_Charges_Total"].isnull()]

# Mostrar amostra dessas linhas (e colunas relevantes)
print("Linhas com 'account_Charges_Total' faltante:")
print(linhas_faltantes[["Churn", "customer_tenure", "account_Contract", "account_Charges_Monthly", "account_Charges_Total"]].head(11))

# Verificar se há padrão (ex: são clientes novos? cancelados?)
print("\nContagem por status:")
print(linhas_faltantes["Churn"].value_counts())

In [ ]:
# Remove as linhas com "account_Charges_Total" igual a NaN.
dados_normal = dados_normal.dropna(subset=["account_Charges_Total"])

# Verificação
print(f"Linhas restantes: {len(dados_normal)}")
print(f"Novos nulos em 'account_Charges_Total': {dados_normal['account_Charges_Total'].isnull().sum()}")

In [ ]:
# Conferir como ficou o dataframe depois do ajuste em "account_Charges_Total".
print(dados_normal.info())


## 2 - Padronização e Transformação dos dados.

* 1 - Transformar as colunas binárias.
* 2 - Transformar as colunas terciárias.
* 3 - Alterar o nome das colunas (variáveis).
* 4 - Atualizar o dicionário de dados.

In [ ]:
# Segunda conferência valores únicos por coluna.
print("\n--- Valores únicos por coluna ---")

for coluna in dados_normal.columns:
    valores_unicos = dados_normal[coluna].nunique()
    print(f'\n* Coluna "{coluna}": {valores_unicos}')
    if valores_unicos < 10:
      print(dados_normal[coluna].unique())

    print("-" * 60)

### 2.1 - Transformar Colunas Binárias.

*Critério*: padrão utilizado para algoritmos de Machine Learning.

| <div align="left">Condição</div> | <div align="left">Codificação</div> |  
|----------------------------------|-------------------------------------|    
|`No` (Não)     | `0` |  
|`Yes` (Sim)    | `1` |

In [ ]:
# Dicionário de mapeamento (padrão ML).
mapeamento_binario = {
    "Yes": 1,    # Evento de interesse = 1
    "No": 0
}

# Aplicar para as colunas binárias:
colunas_binarias = ["Churn", "customer_Partner", "customer_Dependents",
                    "phone_PhoneService", "account_PaperlessBilling"]
for coluna in colunas_binarias:
    if coluna in dados_normal.columns:
        dados_normal[coluna] = dados_normal[coluna].replace(mapeamento_binario)

# Verificação
print(dados_normal[colunas_binarias].head())

### 2.2 - Transformar Colunas Terciárias.

*Critério*: codificação ordinal pela dependência hierárquica das variáveis que originam cada coluna em tratamento.

| <div align="left">Condição</div> | <div align="left">Codificação</div> |  
|----------------------------------|---------------------------------------|  
|`phone_PhoneService = 0` (No)     | `phone_MultipleLines = 0` |  
|`phone_PhoneService = 1` (Yes)    | `phone_MultipleLines = 1` ou `2` |



In [ ]:
# Transformar colunas terciárias.
terciarias = {
    "phone_MultipleLines": {"No phone service": 0, "No": 1, "Yes": 2}
}

# Aplica todas as transformações.
for coluna, mapeamento in terciarias.items():
    dados_normal[coluna] = dados_normal[coluna].map(mapeamento)

# Verifica consistência.
for coluna in terciarias:
    binaria_correspondente = "phone_PhoneService" if "phone" in coluna else "internet_Service"
    print(f"\nConsistência entre {coluna} e {binaria_correspondente}:")
    print(dados_normal.groupby(binaria_correspondente)[coluna].value_counts())

Variável "InternetService" e suas derivadas: "internet_OnlineSecurity", "internet_OnlineBackup", "internet_DeviceProtection", "internet_TechSupport", "internet_StreamingTV", "internet_StreamingMovies".

*Critério*: codificação ordinal pela dependência hierárquica das variáveis que originam cada coluna em tratamento.


In [ ]:
# Mapeamento principal:
mapeamento_internet = {
    "No": 0,              # Não tem internet.
    "DSL": 1,             # Internet básica (DSL).
    "Fiber optic": 2      # Internet premium (Fibra).
}

# Aplica transformação
dados_normal["internet_InternetService"] = dados_normal["internet_InternetService"].map(mapeamento_internet)

# Verificação
print("\nContagem após transformação:")
print(dados_normal["internet_InternetService"].value_counts())



In [ ]:
# Checar se os clientes que não tem Internet Service (0), tem Phone Service (1)

# Filtra clientes SEM internet (internet_InternetService = 0)
clientes_sem_internet = dados_normal[dados_normal["internet_InternetService"] == 0]

# Contagem do serviço de telefone nesse grupo
print("Clientes SEM internet - Contagem de phone_PhoneService:")
print(clientes_sem_internet["phone_PhoneService"].value_counts())

# Verifica se há inconsistências (ex: phone_PhoneService = 1 mas internet = 0)
inconsistencias = dados_normal[
    (dados_normal["internet_InternetService"] == 0) &
    (dados_normal["phone_PhoneService"] == 1)  # Tem telefone mas não tem internet?
]

print("\nLinhas inconsistentes (internet = 0 mas phone = 1):", len(inconsistencias))
if len(inconsistencias) > 0:
    print(inconsistencias[["internet_InternetService", "phone_PhoneService"]].head())

**É possível um cliente ter phone_PhoneService = 1 (tem telefone) mas internet_InternetService = 0 (não tem internet)?**

Se não for possível, essas linhas devem ser corrigidas/excluídas.

Se for possível, documentar a regra no dicionário de dados.

In [ ]:
# Continuar transformação das colunas terciárias.
# Lista de colunas terciárias derivadas.
colunas_terciarias = [
    "internet_OnlineSecurity",
    "internet_OnlineBackup",
    "internet_DeviceProtection",
    "internet_TechSupport",
    "internet_StreamingTV",
    "internet_StreamingMovies"
]

# Aplica o mapeamento padrão e verifica dependência.
for coluna in colunas_terciarias:
    dados_normal[coluna] = dados_normal[coluna].map({
        "No internet service": 0,
        "No": 1,
        "Yes": 2
    })

    # Checa consistência com internet_InternetService.
    inconsistencias = dados_normal[
        (dados_normal["internet_InternetService"] == 0) &
        (dados_normal[coluna] != 0)
    ]
    if not inconsistencias.empty:
        print(f"{len(inconsistencias)} inconsistências em {coluna}. Corrija antes de prosseguir!")

### 2.3 - Transformar Coluna "account_Contract".

*Critério*: codificação numérica ordinal

| Valor Original      | Código Numérico | Classificação de Fidelidade         |
|---------------------|:---------------:|------------------------------------|
| `Month-to-month`    | 0               | Short Term (Menos Fidelização)     |
| `One year`          | 1               | Medium Term (Fidelidade Média)     |
| `Two year`          | 2               | Long Term (Fidelidade Máxima)      |

In [ ]:
# TRANSFORMAÇÃO DA VARIÁVEL "account_Contract".
# Mapeamento para valores numéricos ordinais
mapeamento_contrato = {
    "Month-to-month": 0,  # Menos Fidelização - Short Term.
    "One year": 1,        # Fidelidade Média - Medium Term.
    "Two year": 2         # Fidelidade Máxima - Long Term.
}

# Aplica a transformação
dados_normal["account_Contract"] = dados_normal["account_Contract"].map(mapeamento_contrato)

# Verificação
print("\nDistribuição dos contratos após transformação:")
print(dados_normal["account_Contract"].value_counts().sort_index())

In [ ]:
total = dados_normal["account_Contract"].sum()
percentuais = dados_normal["account_Contract"].value_counts(normalize=True) * 100
percentuais

In [ ]:
# Calcular total de Churn por tipo de contrato
churn_por_contrato = dados_normal[dados_normal["Churn"] == 1]["account_Contract"].value_counts().sort_index()

# Resultado:
print("Total de Churn por tipo de contrato:")
print(churn_por_contrato)

In [ ]:
churn_do_periodo = dados_normal["Churn"].sum()
churn_do_periodo

### 2.4 - Transformar Coluna "account_PaymentMethod".

*Critério*: codificação numérica ordinal baseada no grau de automação do pagamento.

| Método de Pagamento (Original)   | Código Numérico | Classificação        | Justificativa                                      |
|----------------------------------|:---------------:|:-------------------:|---------------------------------------------------|
| `Mailed check`                   | 0               | Manual              | Pagamento físico por correio.                      |
| `Electronic check`               | 1               | Semi-automático     | Digital mas requer ação do cliente.                |
| `Bank transfer (automatic)`      | 2               | Automático          | Débito automático via banco.                       |
| `Credit card (automatic)`        | 3               | Automático Premium  | Pagamento automático com benefícios de cartão.     |


In [ ]:
# TRANSFORMAÇÃO DA VARIÁVEL "account_PaymentMethod".
# Mapeamento para valores ordinais (0 a 3)
mapeamento_pagamento = {
    'Mailed check': 0,               # Menos automático
    'Electronic check': 1,
    'Bank transfer (automatic)': 2,
    'Credit card (automatic)': 3     # Mais automático
}

dados_normal["account_PaymentMethod"] = dados_normal["account_PaymentMethod"].map(mapeamento_pagamento)

# Verificação rápida
print("\nDistribuição após transformação:")
print(dados_normal["account_PaymentMethod"].value_counts().sort_index())

In [ ]:
# Conferir método de pagamento com short term contract.
# Filtrar só clientes month-to-month (account_Contract == 0)
clientes_mes_a_mes = dados_normal[dados_normal["account_Contract"] == 0]

# Contagem de métodos de pagamento nesse grupo
distribuicao_pagamento = clientes_mes_a_mes["account_PaymentMethod"].value_counts().sort_index()

# Taxa de Churn por método de pagamento (nesse grupo)
taxa_churn = clientes_mes_a_mes.groupby("account_PaymentMethod")["Churn"].mean()

# Resultado
print("Distribuição de Pagamento (Month-to-Month):")
print(distribuicao_pagamento)
print("\nTaxa de Churn por Método (Month-to-Month):")
print(taxa_churn)

In [ ]:
# Conferência dos valores por variável, após as transformações.
print("\n--- Valores únicos por coluna ---")

for coluna in dados_normal.columns:
    valores_unicos = dados_normal[coluna].nunique()
    print(f'\n* Coluna "{coluna}": {valores_unicos}')
    if valores_unicos < 10:
      print(dados_normal[coluna].unique())

    print("-" * 60)

### 2.5 - Alterar Nome das Colunas, para agilizar algoritmos de Machine Learning.

Fazer alteração dos nomes das variáveis para melhor entendimento no processo de Machine Learning.



In [ ]:
# Dicionário de mapeamento (versão otimizada para ML)
rename_dict = {
    'customerID': 'customer_id',
    'Churn': 'target_churn',
    'customer_gender': 'gender',
    'customer_SeniorCitizen': 'is_elderly',
    'customer_Partner': 'has_partner',
    'customer_Dependents': 'has_dependents',
    'customer_tenure': 'tenure_months',
    'phone_PhoneService': 'has_phone_service',
    'phone_MultipleLines': 'has_multiple_lines',
    'internet_InternetService': 'internet_service_type',
    'internet_OnlineSecurity': 'has_online_security',
    'internet_OnlineBackup': 'has_online_backup',
    'internet_DeviceProtection': 'has_device_protection',
    'internet_TechSupport': 'has_tech_support',
    'internet_StreamingTV': 'uses_streaming_tv',
    'internet_StreamingMovies': 'uses_streaming_movies',
    'account_Contract': 'contract_term_months',
    'account_PaperlessBilling': 'uses_paperless_billing',
    'account_PaymentMethod': 'payment_method_type',
    'account_Charges_Daily': 'daily_charges',
    'account_Charges_Monthly': 'monthly_charges',
    'account_Charges_Total': 'total_charges'
}

# Aplicar a renomeação
dados_final = dados_normal.rename(columns = rename_dict)

# Verificação rápida
print("Colunas Renomeadas, ver amostra:")
print(dados_final.columns.tolist()[:5])  # Exibe as 5 primeiras para verificação


In [ ]:
# Lista colunas originais que não foram mapeadas (deve retornar vazio)
colunas_nao_mapeadas = set(dados_final.columns) & set(rename_dict.keys())
if colunas_nao_mapeadas:
    print(f"Atenção: Estas colunas não foram renomeadas: {colunas_nao_mapeadas}")
else:
    print("Todas as colunas foram renomeadas com sucesso!")


### 2.6 - Gerar arquivo .csv para próxima etapa.

In [ ]:
caminho_saida = Path("..") / "data" / "processed" / "dados_final.csv"
caminho_saida.parent.mkdir(parents=True, exist_ok=True)
dados_final.to_csv(caminho_saida, index = False)

## 3 - Visualizações para Analisar o Churn da Telecom X.

### 3.1 - Taxa de Churn da Telecom X.

In [ ]:
# Calcula a taxa global de Churn.
total_clientes = len(dados_final)
total_churn = dados_final["target_churn"].sum()
taxa_global = (total_churn / total_clientes) * 100

# Gráfico Simples
plt.figure(figsize = (8, 5))
bars = plt.bar(
    ["Clientes No Churn", "Clientes que Deram Churn"],
    [(total_clientes - total_churn), total_churn],
    color = ["#3498db", "#e74c3c"]
)

# Anotações
plt.annotate(
    f"Taxa Global de Churn: {taxa_global:.1f}%",
    xy = (1, total_churn),
    xytext = (-5 , 20),
    textcoords = "offset points",
    fontsize = 10,
    ha ="center"
)

# Customização
plt.title("TAXA DE CHURN NO PERÍODO - TELECOM X", fontsize = 14, pad = 15)
plt.ylabel("Número de Clientes", fontsize = 11)
plt.grid(axis = "y", linestyle = ":", alpha = 0.4)
sns.despine()

# Adiciona valores nas barras
for bar in bars:
    height  =  bar.get_height()
    plt.text(
        bar.get_x() + bar.get_width() / 2.,
        height,
        f"{height:,}".replace(",", "."),
        ha = "center",
        va = "bottom",
        fontsize = 11
    )

# Salvar gráfico como PNG.
caminho_grafico = Path("..") / "reports" / "churn_taxa_periodo.png"
caminho_grafico.parent.mkdir(parents=True, exist_ok=True)
plt.savefig(caminho_grafico, dpi=300, bbox_inches="tight")

plt.show()

### 3.2 - Tempo de Permanência Clientes Churn.

In [ ]:
# Configuração inicial.
plt.figure(figsize = (12, 6))

# Histograma + KDE
sns.histplot(
    data = dados_final,
    x = "tenure_months",
    hue = "target_churn",
    bins = 24,
    palette = {0: "#2ecc71", 1: "#e74c3c"},
    kde = True,
    alpha = 0.7,
    multiple = "stack"
)

# Linhas críticas
plt.axvline(x = 6, color = "gray", linestyle = ":", label = "6 meses (pico de Churn)")
plt.axvline(x = 12, color = "black", linestyle = "--", label = "1 ano (estabilização)")

# Customização
plt.title("DISTRIBUIÇÃO DO CHURN POR TEMPO DE PERMANÊNCIA", fontsize = 14, pad = 15)
plt.xlabel("Meses de Permanência", fontsize = 11)
plt.ylabel("Número de Clientes", fontsize = 11)
plt.legend(title = "Churn", labels = ["Não", "Sim", "6 meses", "1 ano"])
plt.grid(axis = "y", linestyle = ":", alpha = 0.4)
sns.despine()

# Destaque estatístico
plt.annotate(
    f"55.5% dos cancelamentos ocorrem\nnos primeiros 12 meses",
    xy = (12, dados_final[dados_final["tenure_months"] <= 12]["target_churn"].sum() * 0.5),
    xytext = (20, 40),
    textcoords = "offset points",
    fontsize = 10,
    ha = "left",
    va = "center"
)

# Salvar gráfico como PNG.
caminho_grafico = Path("..") / "reports" / "churn_tempo_permanencia.png"
caminho_grafico.parent.mkdir(parents=True, exist_ok=True)
plt.savefig(caminho_grafico, dpi=300, bbox_inches="tight")

plt.show();

In [ ]:
# Memória de cálculo para cancelamento nos primeiros 12 meses.
# 1. Filtra clientes com tenure <= 12 meses
clientes_ate_12_meses = dados_final[dados_final["tenure_months"] <= 12]

# 2. Soma os casos de Churn nesse grupo
total_churn_12m = clientes_ate_12_meses["target_churn"].sum()

# 3. Calcula a porcentagem em relação ao Churn total
porcentagem = (total_churn_12m / dados_final["target_churn"].sum()) * 100

# Resultado:
print(f"Churn total: {dados_final['target_churn'].sum()} clientes")
print(f"Churn em contratos <=12 meses: {total_churn_12m} clientes ({porcentagem:.1f}%)")

### 3.3 - Clientes Churn versus Tipo de Contrato.

In [ ]:
# Configurações iniciais
plt.figure(figsize=(10, 6))

# Pré-agrega os dados para evitar divisão por zero
dados_agg = (dados_final.groupby('contract_term_months')['target_churn']
             .apply(lambda x: sum(x)/len(x)*100 if len(x) > 0 else 0)
             .reset_index())

# Plot com as posições originais mantidas
ax = sns.barplot(
    data=dados_agg,
    x="contract_term_months",
    y="target_churn",
    palette=["#ff6b6b", "#ffd166", "#06d6a0"],
    hue="contract_term_months",
    errorbar=None,
    dodge=False  # Mantém as barras alinhadas
)

plt.gca().legend_.remove()

# Anotações
for p in ax.patches:
    if p.get_height() > 0:
        ax.annotate(
            f"{p.get_height():.1f}%",
            (p.get_x() + p.get_width() / 2., p.get_height()),
            ha="center",
            va="center",
            xytext=(0, 10),
            textcoords="offset points",
            fontsize=11,
            fontweight="bold"
        )

# Destaque (ajuste as coordenadas conforme necessário)
ax.annotate(
    "Churn 15× maior\nem contratos mensais:\n (42,7% / 2,8%).",
    xy=(0, 40),
    xytext=(0.5, 50),
    fontsize=10,
    ha="center"
)

# Customização
plt.title("TAXA DE CHURN POR TIPO DE CONTRATO", fontsize=14, pad=15)
plt.xlabel("Tipo de Contrato", fontsize=11)
plt.xticks(
    ticks=[0, 1, 2],
    labels=["Mensal", "Anual", "Bienal"],
    fontsize=11
)
plt.ylabel("Taxa de Churn (%)", fontsize=11)
plt.ylim(0, 60)
sns.despine()
plt.grid(axis="y", linestyle=":", alpha=0.3)

# Salvar
caminho_grafico = Path("..") / "reports" / "churn_tipo_contrato.png"
caminho_grafico.parent.mkdir(parents=True, exist_ok=True)
plt.savefig(caminho_grafico, dpi=300, bbox_inches="tight")


plt.show();


### 3.4 - Clientes Churn Mensal por Método de Pagamento.

In [ ]:
# Filtrar Churn mensal e calcular distribuição
churn_mensal = dados_final[(dados_final["contract_term_months"]  ==  0) & (dados_final["target_churn"]  ==  1)]
contagem = churn_mensal["payment_method_type"].value_counts().sort_values(ascending = False)
porcentagens = (contagem / len(churn_mensal)) * 100

# Mapear códigos para labels
labels = {
    0: "Cheque via Correio",
    1: "Cheque Eletrônico",
    2: "Débito em Conta",
    3: "Cartão de Crédito"
}

# Cores com destaque para métodos de risco
cores = ["#e74c3c" if metodo in [0, 1] else "#3498db" for metodo in porcentagens.index]

# Plot
plt.figure(figsize = (10, 4))
bars = plt.barh(
    [labels[i] for i in porcentagens.index],
    porcentagens,
    color = cores,
    alpha = 0.8
)

# Anotações
for bar in bars:
    width = bar.get_width()
    plt.text(
        width + 1,
        bar.get_y() + bar.get_height()/2,
        f"{width:.1f}%",
        ha = "left",
        va = "center",
        fontsize = 10
    )

# Destaque de risco
plt.annotate(
    "Métodos com maior potencial\nde inadimplência",
    xy = (porcentagens.iloc[1] + 5, 0.5),
    xytext = (50, 5),
    textcoords = "offset points",
    fontsize = 9,
    color = "#e74c3c"
)

plt.figtext(
    0.5, -0.1,
    "Nota: Análise restrita aos clientes com plano mensal (código 0) que deram Churn.",
    ha="center",
    fontsize=10,
    color="#666666"
)
plt.subplots_adjust(bottom=0.2)

# Customização
plt.title("COMPOSIÇÃO DO CHURN (CLIENTES MENSALISTAS) POR MÉTODO DE PAGAMENTO", fontsize = 12, pad = 12)
plt.xlabel("% do Total de Churn Clientes Mensalistas", fontsize = 10)
plt.xlim(0, porcentagens.max() * 1.3)
sns.despine(left = True)
plt.grid(axis = "x", linestyle = ":", alpha = 0.3)
plt.tight_layout()

# Salvar
caminho_grafico = Path("..") / "reports" / "churn_mes_tp_pgto.png"
caminho_grafico.parent.mkdir(parents=True, exist_ok=True)
plt.savefig(caminho_grafico, dpi=300, bbox_inches="tight")

plt.show()

#### Nota Referente ao Total de Churn da Telecom X.

  "O **Cheque Eletrônico** representa **57,3% de TODOS os cancelamentos** da base analisada, sendo que **92,8% desses casos** são de clientes com contratos mensais – um risco operacional evitável com migração para pagamentos automáticos."

***Método de Cálculo***  
- `Nota 1`: (Churn_Eletrônico_Total / Churn_Total) × 100  
- `Nota 2`: (Churn_Eletrônico_Mensalistas / Churn_Mensalistas_Total) × 100  


In [ ]:
# Total de Churn na base
total_churn = dados_final["target_churn"].sum()

# Churn com pgto. via Cheque Eletrônico (código 1)
churn_eletronico_total = dados_final[(dados_final["payment_method_type"] == 1) &
                                   (dados_final["target_churn"] == 1)].shape[0]

# Percentual do Churn total pago com Cheque Eletrônico
percentual_total = (churn_eletronico_total / total_churn) * 100

# Dentro do Churn Eletrônico, quantos são mensais (código 0)
churn_eletronico_mensal = dados_final[(dados_final["payment_method_type"] == 1) &
                                     (dados_final["target_churn"] == 1) &
                                     (dados_final["contract_term_months"] == 0)].shape[0]

percentual_mensal_no_eletronico = (churn_eletronico_mensal / churn_eletronico_total) * 100

print(
    f"IMPACTO DO CHEQUE ELETRÔNICO NO CHURN TOTAL:\n"
    f"Total de cancelamentos: {total_churn} clientes\n"
    f"Cancelamentos Pgto. c/ Cheque Eletrônico: {churn_eletronico_total} ({percentual_total:.1f}% do total)\n"
    f"Desses, {churn_eletronico_mensal} são de contratos mensais ({percentual_mensal_no_eletronico:.1f}% do Churn Eletrônico)"
)


### 3.5 - Comparação Daily Charges para Churn e Não Churn.

In [ ]:
# Configurações Iniciais
plt.figure(figsize=(10, 6))
ax = sns.boxplot(
    data=dados_final,
    x="target_churn",
    y="daily_charges",
    palette={0: "#2ecc71", 1: "#e74c3c"},
    #hue="target_churn",
    showfliers=False  # Remove outliers para melhor visualização
)

# Remove a legenda duplicada manualmente
# ax.legend_remove()  # Esta é a forma correta de remover a legenda no Seaborn

# Personalização adicional
plt.title("DISTRIBUIÇÃO DE CUSTO DIÁRIO POR STATUS DE CHURN", fontsize=14, pad=15)
plt.xlabel("Status do Cliente", fontsize=11)
plt.ylabel("Custo Diário ($)", fontsize=11)
plt.xticks([0, 1], ["Clientes Ativos", "Clientes que Cancelaram"])  # Rótulos mais descritivos
sns.despine()
plt.grid(axis='y', linestyle=':', alpha=0.3)

# Nota de rodapé crítica
plt.figtext(
    0.5, -0.05,
    "Nota 1: Não há informação se contratos mensais têm acréscimo por cancelamento.\n"
    "Nota 2: 'Churn' pode incluir encerramentos naturais de contratos mensais (sem renovação).",
    ha = "center",
    fontsize = 9,
    color = "#666666",
    style = "italic"
)

# Salvar o gráfico
caminho_grafico = Path("..") / "reports" / "custo_dia_status.png"
caminho_grafico.parent.mkdir(parents=True, exist_ok=True)
plt.savefig(caminho_grafico, dpi=300, bbox_inches="tight")

plt.show()

In [ ]:
# Estatísticas descritivas por status de Churn
desc_stats = dados_final.groupby("target_churn")["daily_charges"].describe()
print(desc_stats.to_markdown())

Principais Estatísticas / Insights:

| Métrica       | Não-Churn (0) | Churn (1) | Diferença | Significado                               |
|---------------|--------------:|----------:|----------:|-------------------------------------------|
| **Média**     |         R\$ 2.04 |    R\$ 2.48 |    +21.6% | Clientes que cancelam pagam **mais**.       |
| **Mediana**   |         R\$ 2.15 |    R\$ 2.66 |    +23.7% | Distribuição assimétrica à direita.         |
| **75º Perc.** |         R\$ 2.95 |    R\$ 3.14 |     +6.4% | Top 25% do Churn paga significativamente mais. |

# 4. Relatório do Projeto


## 4.1 Objetivo: Investigar o CHURN DE CLIENTES
Para isso, a empresa nos disponibilizou o arquivo "TelecomX_Data.json", com dados de clientes em um determinado período.

## 4.2 Buscar, tratar e realizar uma análise exploratoria dos dados de cliente.

### 4.2.1 Carregar a base de dados.
A) Dados carregados através de referenciamento ao endereço fornecido no GitHub.

B) Verificou-se que os dados estavam com as variáveis aninhadas.

C) Realizada a normalização da base de dados.


### 4.2.2 Análise Exploratória Inicial
Atividades realizadas na exploração inicial:
* verificação de valores nulos;
* verificação de valores em branco ou vazios;
* verificação de campos com espaços adicionais;
* verificação de outras situações que podem gerar "erros" (duplicados, inconsistências).

Feitas as correções das inconsistências identificadas.

### 4.2.3 Dicionário de Dados
Incorporado ao Jupyter Notebook o dicionário de dados original fornecido pela empresa, para facilitar a compreensão das variáveis.


### 4.2.4 Inclusão da coluna Charges Daily.
Foi feita a inclusão da coluna Charges Daily, como solicitada no escopo do projeto.

*Critério utilizado:* (Charges_Monthly / 30)

O uso da coluna Charges_Total pode trazer distorções para os contratos Long Term, já que carregam a somatória de todo o período do contrato, com possíveis situações diferenciadas do período atual em análise.


## 4.3 Padronização e Transformação dos Dados.
Realizada transformação das variáveis conforme descrito abaixo, renomeação das variáveis e atualização do dicionário de dados.

*Critério:* padrão utilizado para algoritmos de Machine Learning.

* 1 - Transformação das colunas binárias.
* 2 - Transformação das colunas terciárias.
* 3 - Transformação das colunas com mais de 3 informações.
* 4 - Alteração do nome das colunas (variáveis) - snake case.
* 5 - Atualização do dicionário de dados.

## 4.4 Visualizações para Analisar os Dados
Gerar gráficos para analisar os dados de clientes, e obter insights para os stackholders tomarem decisões sobre as situações identificadas.

### 4.4.1 Taxa Geral de Churn da Telecom X.
Calculada em 26,6% sobre a base de clientes analisada.

<img src="../notebooks/churn_taxa_periodo.png" alt="Taxa de Churn" width="800">

### 4.4.2 Distribuição do Churn por Tempo de Permanência na Cia.
Verificou-se que 55.5% dos cancelamentos ocorrem nos primeiros 12 meses.

<img src="../notebooks/churn_tempo_permanencia.png" alt="Churn por Tempo de Permanência" width="800">

### 4.4.3 Clientes que Deram Churn versus Tipo de Contrato.
Verificou-se que o Churn é 15x maior em contratos mensais: (42,7% / 2,8%) = 15,25.

<img src="../notebooks/churn_tipo_contrato.png" alt="Churn por Tipo de Contrato" width="800">

### 4.4.4 Clientes Churn Mensal por Método de Pagamento.
Análise restrita apenas aos clientes mensalistas, foi constatado que 77% dos clientes desta categoria utilizam cheque ou cheque eletrônico como forma de pagamento, que são os **métodos com maior potencial de inadimplência**.

O **Cheque Eletrônico** representa **57,3% de TODOS os cancelamentos** da base analisada, sendo que **92,8% desses casos** são de clientes com contratos mensais – um risco operacional evitável com migração para pagamentos automáticos."

<img src="../notebooks/churn_mes_tp_pgto.png" alt="Churn por Forma de Pagamento" width="800">





### 4.4.5 Comparação Valor Diário para Churn e Não Churn.
A comparação visual demonstra que os clientes Churn pagam valores diários maiores que os clientes que permanecem na cia.

| Métrica       | Não-Churn (0) | Churn (1) | Diferença | Significado                               |
|---------------|--------------:|----------:|----------:|-------------------------------------------|
| **Média**     |         R\$ 2.04 |    R\$ 2.48 |    +21.6% | Clientes que cancelam pagam **mais**.       |
| **Mediana**   |         R\$ 2.15 |    R\$ 2.66 |    +23.7% | Distribuição assimétrica à direita.         |
| **75º Perc.** |         R\$ 2.95 |    R\$ 3.14 |     +6.4% | Top 25% do Churn paga significativamente mais. |

<img src="../notebooks/custo_dia_status.png" alt="Custo Diário por Status de Churn" width="600">

# 5. CONCLUSÃO
Após todas as etapas descritas anteriormente, foi gerado um arquivo de dados limpo e transformado (`"dados_final.csv"`) para que as pessoas analistas de dados possam aplicar o algoritmo de Machine Learning mais apropriado para esta situação.

Outros insights podem ser gerados a partir da base de dados fornecida, mas diante da relevância da situação de churn nos contratos do tipo mensal, recomenda-se uma ação para corrigir as distorções apresentadas, ou pelo menos colher mais informações para mehor esclarecer a frequência das ocorrências de churn nessa categoria de contratos.